In [ ]:
import os

import pyphare.pharein as ph
from pyphare.simulator.simulator import Simulator
from pyphare.pharesee.run import Run

from pyphare.pharein import global_vars
from tests.diagnostic import all_timestamps

from pyphare.pharein.diagnostics import FluidDiagnostics

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

mpl.use("Agg")

In [ ]:
def config():

    sim = ph.Simulation(
        smallest_patch_size=20,
        largest_patch_size=60,
        time_step=0.01,
        time_step_nbr=2,
        boundary_types="periodic",
        cells=100,
        dl=0.2,
        diag_options={
            "format": "phareh5",
            "options": {"dir": "nCheck", "mode": "overwrite"},
        },
    )

    L = sim.simulation_domain()[0]

    def densityMain(x):
        return 1.0

    def densityBeam(x):
        u = x/L-0.5
        return np.exp(-u**2)

    def bx(x):
        return 1.0

    def by(x):
        return 0.0

    def bz(x):
        return 0.0

    def v0(x):
        return 0.0

    def vth(x):
        return np.sqrt(1.0)

    v_pop = {
        "vbulkx": v0,
        "vbulky": v0,
        "vbulkz": v0,
        "vthx": vth,
        "vthy": vth,
        "vthz": vth,
    }

    ph.MaxwellianFluidModel(
        bx=bx,
        by=by,
        bz=bz,
        main={"mass": 2, "charge": 1, "density": densityMain, **v_pop},
        beam={"mass": 3, "charge": 2, "density": densityBeam, **v_pop},
    )

    ph.ElectronModel(closure="isothermal", Te=0.0)

    timestamps = all_timestamps(global_vars.sim)

    for quantity in ["density", "charge_density"]:
        FluidDiagnostics(
            quantity=quantity,
            write_timestamps=timestamps
        )

    poplist = ["main", "beam"]
    for pop in poplist:
        for quantity in ["density", "charge_density", "mass_density"]:
            FluidDiagnostics(
                quantity=quantity,
                write_timestamps=timestamps,
                population_name=pop,
            )

    return sim

In [ ]:
def get_hierarchies(run_path):
    r = Run(run_path)

    N_hier = r.GetNi(0.0)

    return N_hier

In [ ]:
sim = config()

In [ ]:
Simulator(sim).run()

In [ ]:
sim.reset()

In [ ]:
run_path = os.path.join(os.curdir, "nCheck")
time = 0.0

In [ ]:
r = Run(run_path)

In [ ]:
NmHier = r.GetN(time, "main")
NbHier = r.GetN(time, "main")
NiHier = r.GetNi(time)